In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
owid_df = pd.read_csv('../data/energy-data/owid-energy-data.csv')
gov_response = pd.read_csv('../data/covid-policy-dataset/data/timeseries_indices/OxCGRT_timeseries_GovernmentResponseIndex_v1.csv')

In [38]:
cov_compact = pd.read_csv('../data/covid-policy-dataset/data/OxCGRT_compact_national_v1.csv')

In [3]:
gov_response_countries = gov_response[gov_response['Jurisdiction']=='NAT_TOTAL']

In [4]:
gov_response_rotate = gov_response_countries[gov_response_countries['CountryName'].isin(owid_df['country'])].drop(columns=['CountryCode'
                                                                                                          ,'RegionCode'
                                                                                                          ,'RegionName'
                                                                                                          ,'CityCode'
                                                                                                          ,'CityName'
                                                                                                          ,'Jurisdiction']).transpose().reset_index()

In [5]:
gov_response_rotate.columns = gov_response_rotate.iloc[0]

In [6]:
gov_response_rotate = gov_response_rotate.drop(index=0).rename(columns={'CountryName':'Date'}).reset_index(drop=True)

In [7]:
gov_num = gov_response_rotate.iloc[:,1:].apply(pd.to_numeric)

In [8]:
gov_date = gov_response_rotate.iloc[:,0].apply(pd.to_datetime)

In [9]:
gov_num.insert(0,'Date',gov_date)

In [10]:
gov_num['Year'] = gov_num['Date'].dt.year

In [11]:
gov_mean = gov_num.groupby('Year').mean().reset_index().drop(columns='Date')

In [12]:
gov_melt = gov_mean.melt(id_vars=['Year'],value_vars=gov_mean.columns[1:]).rename(columns={0:'Country'})

In [13]:
owid_trim = owid_df[['country',
                     'year',
                     'coal_consumption',
                     'hydro_consumption',
                     'oil_consumption',
                     'nuclear_consumption',
                     'gas_consumption']]

In [14]:
owid_trim = owid_trim[(owid_trim['country'].isin(gov_melt['Country']))&
                    (owid_trim['year'].between(2019,2023))].rename(columns={'country':'Country',
                                                                            'year':'Year'})

In [15]:
gov_owid_merge = owid_trim.merge(gov_melt,how='outer',on=('Country','Year'))

In [36]:
#gov_owid_merge.to_csv('gov_owid_merge.csv',index=False)

In [17]:
gov_owid_merge

,Country,Year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption,value
0,Afghanistan,2019,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2020,NaN,NaN,NaN,NaN,NaN,33.816694
2,Afghanistan,2021,NaN,NaN,NaN,NaN,NaN,27.471233
3,Afghanistan,2022,NaN,NaN,NaN,NaN,NaN,16.957507
4,Afghanistan,2023,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
875,Zimbabwe,2019,NaN,NaN,NaN,0.0,NaN,NaN
876,Zimbabwe,2020,NaN,NaN,NaN,0.0,NaN,44.732240
877,Zimbabwe,2021,NaN,NaN,NaN,0.0,NaN,54.878137
878,Zimbabwe,2022,NaN,NaN,NaN,0.0,NaN,44.476356


In [42]:
cov_compact[cov_compact['CountryName'].isin(gov_owid_merge['Country'])]

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,...,V3_Vaccine Financial Support (summary),V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200101,0.0,NaN,0.0,NaN,...,0,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200102,0.0,NaN,0.0,NaN,...,0,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200103,0.0,NaN,0.0,NaN,...,0,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200104,0.0,NaN,0.0,NaN,...,0,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200105,0.0,NaN,0.0,NaN,...,0,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202755,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,20221227,1.0,1.0,2.0,1.0,...,5,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0
202756,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,20221228,1.0,1.0,2.0,1.0,...,5,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0
202757,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,20221229,1.0,1.0,2.0,1.0,...,5,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0
202758,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,20221230,1.0,1.0,2.0,1.0,...,5,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0


In [46]:
owid_df[['country',
        'year',
        'coal_consumption',
        'hydro_consumption',
        'oil_consumption',
        'nuclear_consumption',
        'gas_consumption']]

,country,year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21807,Zimbabwe,2019,NaN,NaN,NaN,0.0,NaN
21808,Zimbabwe,2020,NaN,NaN,NaN,0.0,NaN
21809,Zimbabwe,2021,NaN,NaN,NaN,0.0,NaN
21810,Zimbabwe,2022,NaN,NaN,NaN,0.0,NaN


In [172]:
owid_sub = owid_trim[(owid_trim['Year'].isin([2019,2021]))&(owid_trim['gas_consumption'].notnull())]

In [148]:
owid_trim_countries = owid_sub[['Country','Year']].drop(columns='Year')

In [176]:
owid_sub_2 = owid_sub.groupby('Country').diff().dropna().drop(columns='Year').reset_index()

In [160]:
owid_sub_2['Country'] = owid_sub['Country'].nunique()

In [92]:
owid_change = owid_df[['country','year','coal_cons_change_pct','gas_cons_change_pct','hydro_cons_change_pct','nuclear_cons_change_pct','oil_cons_change_pct'

In [96]:
owid_change[(owid_change['year']==2020)&(owid_change['country'].isin(]

,country,year,coal_cons_change_pct,gas_cons_change_pct,hydro_cons_change_pct,nuclear_cons_change_pct,oil_cons_change_pct
20,ASEAN (Ember),2020,NaN,NaN,NaN,NaN,NaN
144,Afghanistan,2020,NaN,NaN,NaN,NaN,NaN
267,Africa,2020,-3.325,-2.199,6.630,-25.272,-10.450
326,Africa (EI),2020,-3.325,-2.199,6.630,-25.272,-10.450
370,Africa (EIA),2020,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21302,Western Sahara,2020,NaN,NaN,NaN,NaN,NaN
21424,World,2020,-2.983,-0.948,2.741,-3.817,-9.089
21468,Yemen,2020,NaN,NaN,NaN,NaN,NaN
21684,Zambia,2020,NaN,NaN,NaN,NaN,NaN


In [180]:
owid_sub_2.merge(owid_trim_countries.reset_index(),how='inner',on='index')

,index,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption,Country
0,757,-1.698,-0.377,-15.747,0.000,30.983,Algeria
1,1133,5.928,-21.089,29.059,4.182,-6.248,Argentina
2,1691,-33.945,4.759,-68.284,0.000,-44.813,Australia
3,1857,-3.783,-6.216,-12.986,0.000,1.150,Austria
4,1901,-0.006,-0.783,2.180,0.000,10.282,Azerbaijan
...,...,...,...,...,...,...,...
60,20366,-4.671,-1.513,-172.137,-26.853,-9.471,United Kingdom
61,20490,-213.566,-101.030,-418.698,-94.447,-145.750,United States
62,20819,4.898,-3.924,3.345,0.000,19.277,Uzbekistan
63,20986,-0.323,12.018,-89.298,0.000,24.829,Venezuela
